In [1]:
import findspark
findspark.init()
import pandas as pd

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .master("local") \
     .appName("count") \
     .config('spark.executor.memory', '4G') \
    .config('spark.driver.memory', '8G') \
     .getOrCreate()
df_origin = spark.read.json('./yelp_academic_dataset_review.json')

In [3]:
df = df_origin['user_id', 'business_id', 'stars']
df.first()

Row(user_id=u'msQe1u7Z_XuqjGoqhB0J5g', business_id=u'iCQpiavjjPzJ5_3gPD5Ebg', stars=2)

In [4]:
# read in useful user_id
users_lines = open('./yelp_userids_64575.txt').readlines()
users_set = set([line[:-1] for line in users_lines])

In [5]:
reviews = df.filter(df.user_id.isin(users_set)).toPandas()

In [6]:
# count unique business_id
reviews['business_id'].nunique()

154457

In [7]:
reviews['user_id'].nunique()

64575

In [6]:
# # generate 1000 random test case
# random = reviews.sample(False, 0.0005, seed=1)
# kk = random.collect()
# kk

In [7]:
# # save 1000 test case
# f = open('./test_small.txt','w') 
# lines = [str(row[0]) + "," + str(row[1]) + "," + str(row[2]) + "\n" for row in f]
# f.writelines(lines)
# f.close()

In [8]:
# readin test case
kk = [line[:-1].split(',') for line in open('./test_small.txt').readlines()]
# kk = [line[:-1].split(',') for line in open('./test_small_2.txt').readlines()]
# kk = [line[:-1].split(' ') for line in open('./final_test.txt').readlines()]

In [9]:
users_knn_lines = [item[:-1].split(',') for item in open('./yelp_userid_knn_corrected.txt').readlines()]
user_dict = {}
for line in users_knn_lines:
    if line[0] not in user_dict:
        user_dict[line[0]] = {}
    user_dict[line[0]][line[1]] = float(line[2])
    
# normalize distance to similarity
# min --> 1, max --> 0.5
for k in user_dict.keys():
    v = user_dict[k]
    min_val = min(v.values())
    max_val = max(v.values())
    for key in v.keys():
        value = v[key]
        v[key] = 1 - 0.5 * (value - min_val) / (max_val - min_val)

In [10]:
def compute(user_id, business_id, result):
    pred = 0
    possible_reviews = []
    close_reviews = []
    
    if user_id not in user_dict:
        pred = 0
    else:   
        review_processed = reviews.loc[reviews['business_id'] == business_id]
        for index, row in review_processed.iterrows():
            possible_reviews.append(row)
        friends = user_dict[user_id].keys()
        for review in possible_reviews:
            if review[0] in friends:
                close_reviews.append(review)
        if len(close_reviews) == 0:
            tmp = [float(review[2]) for review in possible_reviews]
            pred = sum(tmp)/len(tmp)
    #         print(user_id, business_id, len(close_reviews), result, int(result) >= 4, pred, pred >= 4, (int(result) >= 4) == (pred >= 4))
        else:
            weight = 0.0
            total = 0.0
            for review in close_reviews:
                weight += user_dict[user_id][review[0]]
                total += float(review[2]) * user_dict[user_id][review[0]]
            pred = int(total / weight)
    #         print(user_id, business_id, len(close_reviews), result, int(result) >= 4, pred, pred >= 4, (int(result) >= 4) == (pred >= 4))

    return (user_id, business_id, len(close_reviews), result, int(result) >= 4, pred, pred >= 4, (int(result) >= 4) == (pred >= 4))
    

In [11]:
rrr = []
correct = 0
total = 0
f = open("./user_based_result.txt", 'w')
for r in kk:
    total += 1
    tmp_rst = compute(r[0], r[1], int(r[2]))
    rrr.append(tmp_rst)
    if tmp_rst[7] is True:
        correct += 1
    ll = [str(item) for item in list(tmp_rst)]
    f.write(','.join([ll[0], ll[1], ll[3], ll[5]]) + "\n")
    lll = [ll[0], ll[1], ll[3], ll[5], ll[7]]
    print(','.join(lll))
f.close()

YHWsLBS8jzZiPjKHMFOaAA,amsvLzfEvCzLwP0MnXAJ1w,4,3.5641025641,False
K1Tc6gxpUvg7IekgL9UfVQ,gUaYljrISaJ1bSEFG3fexw,2,2.0,True
yk8P6x1ksMwCY5aGa-u7Jg,0sE5o_dunOdUMvXVsaJSsA,5,3.16,False
ITtlUIvrlpUDDTueuTpRPw,ietA7hICMrj51YmrCmZG7g,5,5.0,True
I7NlxVcCs2akeluwv2S7lg,_9vGmNrEcLWFOnUzeUdfag,5,5,True
0G9BcXSQscv_GnhlNwFfjw,qmymSqVwHYRqdwfcBatzpQ,2,3,True
OAzlpeH3UnzGbmWZZbbRnw,LEYZb8jiD36RQTElxy0YrQ,2,3.45762711864,True
J8fPgOsnir5Lmu0eYEKCVA,FSB3Jb4t11KkMyEi_7-s6Q,5,3.84285714286,False
uLQJAaULt91a263PsRyEqQ,68PJJkcq_i0SlLqO6t7Qxw,5,3.59433962264,False
bHaVRKoLsWcq0E-drLfR3w,s_CsWzEMOlk6WVCEHjk-1w,5,5.0,True
xovKY9oB8s-NjKz_I1RpHQ,IsN0qEzgAXBjbT3qn6lokA,4,3.66949152542,False
ELfzWgdf64VBLi5z1ECItw,wke61EJKd1Yw6q1BR1npZw,5,2,False
DOTzP2qNughplMwJeoPdwg,Avr_qHllB60Ade8IwnU7Nw,5,4.2,True
tWJrrj2WUCCU5l6DwvRnOQ,-o0xWAEO-C2oqlyt9TO8qg,4,3.76,False
HikPdCQGk1mk5JsPjur7Nw,ZMbY3xmTJ3EB2f1ZFyB5Dg,3,2.33333333333,True
5tP46vxV14U5XrcrqUoiLw,PeAVCjePbJ54UYZ5l4iV7Q,5,4.63157894737,True
U_Ne8BOcLn1iWBgI

In [19]:
rows_1 = [line[:-1].split(",") for line in open('./user_based_result.txt').readlines()]
rows_2 = [line[:-1].split(",") for line in open('./user_based_result_2.txt').readlines()]

In [20]:
# accuracy
# 正确True的个数 / 总数
def computeResult(rows):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for row in rows:
        rst = float(row[2]) >= 4
        pred = float(row[3]) >= 4
        if rst and pred:
            TP += 1
        elif not rst and not pred:
            TN += 1
        elif not rst and pred: # false positive
            FP += 1
        elif rst and not pred: # false negative
            FN += 1

    accuracy = 1.0 * (TP + TN) / (TP + TN + FP + FN) * 100
    precision = 1.0 * (TP) / (TP + FP) * 100
    recall = 1.0 * (TP) / (TP + FN) * 100
    print("accuracy: %3.2f%%" % accuracy)
    print("precision: %3.2f%%" % precision)
    print("recall: %3.2f%%" % recall)

In [21]:
computeResult(rows_1)
computeResult(rows_2)

accuracy: 63.38%
precision: 83.12%
recall: 56.06%
accuracy: 61.49%
precision: 79.69%
recall: 54.74%


In [23]:
# compute RMSE
import math
rows_1 = [line[:-1].split(",") for line in open('./user_based_result.txt').readlines()]
summary = 0
length = len(rows_1)
for row in rows_1:
    summary += (float(row[2]) - float(row[3])) * (float(row[2]) - float(row[3]))
RMSE = math.sqrt(summary/length)
print(RMSE)

1.0874844523
